In [46]:
import pandas as pd


train=pd.read_csv("electric_train.csv")


# drop first column
train.drop(train.columns[0], axis=1, inplace=True)

In [69]:
test=pd.read_csv("electric_test.csv")
# drop first column
test.drop(test.columns[0], axis=1, inplace=True)

In [48]:
train.columns
for idx, col in enumerate(train.columns):
    print(idx, col)
    train.rename(columns={col: col.replace("electric_train.", "")}, inplace=True)

0 electric_train.num
1 electric_train.tm
2 electric_train.hh24
3 electric_train.n
4 electric_train.stn
5 electric_train.sum_qctr
6 electric_train.sum_load
7 electric_train.n_mean_load
8 electric_train.nph_ta
9 electric_train.nph_hm
10 electric_train.nph_ws_10m
11 electric_train.nph_rn_60m
12 electric_train.nph_ta_chi
13 electric_train.weekday
14 electric_train.week_name
15 electric_train.elec


In [49]:
train.columns

Index(['num', 'tm', 'hh24', 'n', 'stn', 'sum_qctr', 'sum_load', 'n_mean_load',
       'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', 'weekday',
       'week_name', 'elec'],
      dtype='object')

In [50]:
# split datetime tm into date and time
train['hour'] = pd.to_datetime(train['tm']).dt.hour   
train['month'] = pd.to_datetime(train['tm']).dt.month
train['day'] = pd.to_datetime(train['tm']).dt.day

In [51]:
train.drop('tm', axis=1, inplace=True)

In [52]:
# I want to change hour into continous variable, since 23 and 0 are close to each other
train

,num,hh24,n,stn,sum_qctr,sum_load,n_mean_load,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,week_name,elec,hour,month,day
0,4821,1,11,884,6950,751.32,68.606449,2.2,62.7,1.8,0.0,-1.0,4,0,99.56,1,1,1
1,4821,2,11,884,6950,692.60,68.606449,2.3,63.1,2.1,0.0,-0.6,4,0,91.78,2,1,1
2,4821,3,11,884,6950,597.48,68.606449,2.2,62.4,2.5,0.0,-1.3,4,0,79.17,3,1,1
3,4821,4,11,884,6950,553.48,68.606449,1.7,63.5,1.7,0.0,-0.2,4,0,73.34,4,1,1
4,4821,5,11,884,6950,526.24,68.606449,1.7,63.0,1.6,0.0,-0.8,4,0,69.73,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,34200,6779.84,225.462000,2.7,46.3,3.1,0.0,-0.4,5,1,130.74,20,12,31
7593351,20947,21,23,671,34200,6802.40,225.462000,2.6,46.8,3.1,0.0,-0.5,5,1,131.18,21,12,31
7593352,20947,22,23,671,34200,6706.68,225.462000,2.4,47.4,2.1,0.0,0.2,5,1,129.33,22,12,31
7593353,20947,23,23,671,34200,6355.88,225.462000,2.5,47.0,2.1,0.0,0.3,5,1,122.57,23,12,31


In [53]:
train.drop(columns=['sum_qctr','sum_load','n_mean_load'], inplace=True)
# take log on nph_ta , nph_hm, nph_ws_10m, nph_rn_60m, nph_ta_chi
# take log and if values are negative take log(abs) and take netaive again,  and if values are 0, take 0
# and add new columns with _log
import numpy as np
for col in ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']:
    train[col+'_log'] = np.log(train[col].abs()+1)
    train[col+'_log'] = np.where(train[col+'_log']<0, -train[col+'_log'], train[col+'_log'])
    train[col+'_log'] = np.where(train[col+'_log']==0, 0, train[col+'_log'])



In [54]:
# if there is any value from train['elec'] is negative, drop that row
train = train[train['elec']>=0]

# I want to add and temperature of hour before
train['nph_ta_before'] = train['nph_ta_chi'].shift(1)
# add log ta_before
train['nph_ta_before_log'] = np.where(train['nph_ta_before']<0, -train['nph_ta_before'], train['nph_ta_before'])



/tmp/ipykernel_1980009/1806122367.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['nph_ta_before'] = train['nph_ta_chi'].shift(1)
/tmp/ipykernel_1980009/1806122367.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['nph_ta_before_log'] = np.where(train['nph_ta_before']<0, -train['nph_ta_before'], train['nph_ta_before'])


In [55]:
# first row has NaN value, so drop that row
train = train.dropna()


In [56]:
train

,num,hh24,n,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,...,hour,month,day,nph_ta_log,nph_hm_log,nph_ws_10m_log,nph_rn_60m_log,nph_ta_chi_log,nph_ta_before,nph_ta_before_log
1,4821,2,11,884,2.3,63.1,2.1,0.0,-0.6,4,...,2,1,1,1.193922,4.160444,1.131402,0.0,0.470004,-1.0,1.0
2,4821,3,11,884,2.2,62.4,2.5,0.0,-1.3,4,...,3,1,1,1.163151,4.149464,1.252763,0.0,0.832909,-0.6,0.6
3,4821,4,11,884,1.7,63.5,1.7,0.0,-0.2,4,...,4,1,1,0.993252,4.166665,0.993252,0.0,0.182322,-1.3,1.3
4,4821,5,11,884,1.7,63.0,1.6,0.0,-0.8,4,...,5,1,1,0.993252,4.158883,0.955511,0.0,0.587787,-0.2,0.2
5,4821,6,11,884,1.7,61.8,0.8,0.0,0.0,4,...,6,1,1,0.993252,4.139955,0.587787,0.0,0.000000,-0.8,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,20,23,671,2.7,46.3,3.1,0.0,-0.4,5,...,20,12,31,1.308333,3.856510,1.410987,0.0,0.336472,-0.4,0.4
7593351,20947,21,23,671,2.6,46.8,3.1,0.0,-0.5,5,...,21,12,31,1.280934,3.867026,1.410987,0.0,0.405465,-0.4,0.4
7593352,20947,22,23,671,2.4,47.4,2.1,0.0,0.2,5,...,22,12,31,1.223775,3.879500,1.131402,0.0,0.182322,-0.5,0.5
7593353,20947,23,23,671,2.5,47.0,2.1,0.0,0.3,5,...,23,12,31,1.252763,3.871201,1.131402,0.0,0.262364,0.2,0.2


In [57]:
# convert elec into log
# train['elec'] = np.log(train['elec']+1)

In [58]:
train['elec']

1           91.78
2           79.17
3           73.34
4           69.73
5           71.29
            ...  
7593350    130.74
7593351    131.18
7593352    129.33
7593353    122.57
7593354    115.12
Name: elec, Length: 7593349, dtype: float64

In [59]:
from autogluon.tabular import TabularDataset, TabularPredictor

#train = pd.read_csv('../preprocess/q1_train_data_2.csv')
#test = pd.read_csv('../preprocess/q1_test_data_2.csv')

train_data = TabularDataset(train)
#test_data = TabularDataset(test)

print("==================Tabular_complete========================")

hyperparameters = {
	'GBM': [{}],
}

save_path = './models'
predictor = TabularPredictor(label='elec', problem_type = 'regression', eval_metric='mean_squared_error',
                            path = save_path)
predictor.fit(train_data, hyperparameters=hyperparameters)

print("==================learning_complete========================")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2
CPU Count:          64
Memory Avail:       99.13 GB / 125.60 GB (78.9%)
Disk Space Avail:   238.32 GB / 467.89 GB (50.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setti

==================Tabular_complete========================


Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    101608.77 MB
	Train Data (Original)  Memory Usage: 1129.69 MB (1.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 12 | ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi', ...]
		('int', [])   :  9 | ['num', 'hh24', 'n', 'stn', 'weekday', ...]
	Types of features in processed da

[1000]	valid_set's l2: 22.9012
[2000]	valid_set's l2: 17.9651
[3000]	valid_set's l2: 15.3645
[4000]	valid_set's l2: 13.6035
[5000]	valid_set's l2: 12.2676
[6000]	valid_set's l2: 11.2638
[7000]	valid_set's l2: 10.478
[8000]	valid_set's l2: 9.7857
[9000]	valid_set's l2: 9.24177
[10000]	valid_set's l2: 8.72411


	-8.7241	 = Validation score   (-mean_squared_error)
	459.65s	 = Training   runtime
	1.13s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBM': 1.0}
	-8.7241	 = Validation score   (-mean_squared_error)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 476.49s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 67447.6 rows/s (75934 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("./models")


==================learning_complete========================


In [60]:
predictor.leaderboard(train_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,-8.435057,-8.724106,mean_squared_error,118.499281,1.125003,459.652542,118.499281,1.125003,459.652542,1,True,1
1,WeightedEnsemble_L2,-8.435057,-8.724106,mean_squared_error,118.543840,1.125821,459.654780,0.044559,0.000819,0.002238,2,True,2


In [61]:
predictor.feature_importance(train_data)

Computing feature importance via permutation shuffling for 21 features using 5000 rows with 5 shuffle sets...
	17.9s	= Expected runtime (3.58s per shuffle set)
	11.77s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
hh24,494.076203,15.472916,1.152727e-07,5,525.935150,462.217255
hour,80.634227,2.716260,1.542880e-07,5,86.227045,75.041410
month,79.406035,2.949332,2.279625e-07,5,85.478750,73.333320
nph_ta_chi,72.414652,1.895448,5.627640e-08,5,76.317406,68.511899
nph_ta_before,52.927624,1.530003,8.370219e-08,5,56.077920,49.777327
weekday,32.608473,1.150664,1.857514e-07,5,34.977705,30.239241
day,23.055279,1.042336,4.999739e-07,5,25.201462,20.909096
num,14.050549,0.423399,9.882834e-08,5,14.922333,13.178765
nph_ta,13.186045,0.187894,4.946062e-09,5,13.572922,12.799168
nph_hm,8.326911,0.831189,1.175703e-05,5,10.038341,6.615481


In [70]:
for idx, col in enumerate(test.columns):
    print(idx, col)
    test.rename(columns={col: col.replace("electric_test.", "")}, inplace=True)

0 electric_test.num
1 electric_test.tm
2 electric_test.hh24
3 electric_test.stn
4 electric_test.nph_ta
5 electric_test.nph_hm
6 electric_test.nph_ws_10m
7 electric_test.nph_rn_60m
8 electric_test.nph_ta_chi
9 electric_test.weekday
10 electric_test.week_name


In [71]:
test['hour'] = pd.to_datetime(test['tm']).dt.hour
test['month'] = pd.to_datetime(test['tm']).dt.month
test['day'] = pd.to_datetime(test['tm']).dt.day
test['n']=0

In [72]:
# take log on test 
for col in ['nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']:
    test[col+'_log'] = np.log(test[col].abs()+1)
    test[col+'_log'] = np.where(test[col+'_log']<0, -test[col+'_log'], test[col+'_log'])
    test[col+'_log'] = np.where(test[col+'_log']==0, 0, test[col+'_log'])

In [65]:
test.drop('tm', axis=1, inplace=True)

In [73]:
test['nph_ta_before'] = test['nph_ta_chi'].shift(1)

# for first row, fill nph_ta_before with nph_ta
test['nph_ta_before'] = np.where(test['nph_ta_before'].isnull(), test['nph_ta'], test['nph_ta_before'])
test['nph_ta_before_log'] = np.where(test['nph_ta_before']<0, -test['nph_ta_before'], test['nph_ta_before'])

In [74]:
test

,num,tm,hh24,stn,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,...,month,day,n,nph_ta_log,nph_hm_log,nph_ws_10m_log,nph_rn_60m_log,nph_ta_chi_log,nph_ta_before,nph_ta_before_log
0,2385,2023-01-01 01:00:00,1,303,7.8,61.5,6.7,0.0,4.2,6,...,1,1,0,2.174752,4.135167,2.041220,0.0,1.648659,7.8,7.8
1,2385,2023-01-01 02:00:00,2,303,7.9,60.6,7.6,0.0,4.0,6,...,1,1,0,2.186051,4.120662,2.151762,0.0,1.609438,4.2,4.2
2,2385,2023-01-01 03:00:00,3,303,8.2,61.9,8.7,0.0,4.1,6,...,1,1,0,2.219203,4.141546,2.272126,0.0,1.629241,4.0,4.0
3,2385,2023-01-01 04:00:00,4,303,8.4,60.9,9.2,0.0,4.3,6,...,1,1,0,2.240710,4.125520,2.322388,0.0,1.667707,4.1,4.1
4,2385,2023-01-01 05:00:00,5,303,8.5,60.9,9.5,0.0,4.3,6,...,1,1,0,2.251292,4.125520,2.351375,0.0,1.667707,4.3,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838234,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,...,12,31,0,1.887070,4.273884,1.435085,0.0,1.916923,5.7,5.7
2838235,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,...,12,31,0,1.840550,4.249923,1.526056,0.0,1.704748,5.8,5.8
2838236,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,...,12,31,0,1.808289,4.271095,1.386294,0.0,1.808289,4.5,4.5
2838237,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,...,12,31,0,1.824549,4.258446,1.568616,0.0,1.808289,5.1,5.1


In [75]:
test_data = TabularDataset(test)
y_pred = predictor.predict(test_data)

# 느낀점
1. 데이터 사이즈가 커서 automl 쓰기 힘들수도있을 것 같다
2. EDA를 해보진 않았는데, 데이터에 대한 이해가 좀더 필요할 것 같다
3. 대부분 수치형데이터라 상관관계 등을 고려한 분석이 이어지면 좋을 것 같다. 


In [76]:
y_pred.to_csv('electric_result_before.csv')

In [ ]:
test.columns

Index(['num', 'hh24', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m',
       'nph_ta_chi', 'weekday', 'week_name', 'hour', 'month', 'day', 'n',
       'nph_ta_log', 'nph_hm_log', 'nph_ws_10m_log', 'nph_rn_60m_log',
       'nph_ta_chi_log', 'nph_ta_before'],
      dtype='object')